In [1]:
using JuMP, Gurobi, Test, Combinatorics, LightGraphs, TimerOutputs

# function solveMP()
const gurobi_env = Gurobi.Env()
# setparams!(gurobi_env, OutputFlag = 0)

Academic license - for non-commercial use only - expires 2022-12-04


Gurobi.Env(Ptr{Nothing} @0x0000000039489880, false, 0)

In [155]:
ins = 4#ARGS[1] #:10 #2,3, do 4 first, 5
dataSet = "N50_d40_"
println("Running...")
global prob = "" #"L"
#     include("./TestInstances/"*fileName*".jl")
global q =[]
include("./TestInstances/"*dataSet*string(ins)*".jl")
global numArcs = length(arcs[:,1])
global lb = 1
global ub = 999
# println("1")
function setQ(prob)
    global lb, ub, numArcs, q
    if prob == "L"
        lb = 1
        ub = 333
    elseif prob == "M"
        lb = 334
        ub = 666
    elseif prob == "H"
        lb = 667
        ub = 999
    end
    new_q = rand(lb:ub, numArcs)./1000
       
    return new_q
end
# println("2")
if prob != ""
    q = setQ(prob)
end
# println("q = ", q)
# q = [0.28, 0.23, 0.033, 0.257, 0.208, 0.037, 0.009, 0.269, 0.302, 0.167, 0.098, 0.288, 0.187]
# println("3")
#     global numArcs, arcs, d_x, q, b_x, origin, destination, R, d_y, b_y
#     global newP, newM, P_set, M_set, numPaths,numY
    
    include("./functionEnumFeasX.jl")
    include("./functionFindQ.jl")
    include("./functionGetCost_SP.jl")

    include("./functionShortestPathBellmanFord.jl")
    include("./functionGenMonitoring.jl")
    include("./functionLP_findM.jl")
    

        
#     global X_feas = EnumX(arcs, b_x, numArcs, d_x)
#     global EnumY = EnumX(arcs, b_y, numArcs, d_y)
    M_start = findall(d_x.<=b_x)
    global M_set = [[M_start[1]]]#[EnumY[1]] #EnumX(arcs, b_y, numArcs, d_y)
    global numY = 1 
    global R = -sum(log10(1-q_) for q_ in q)  #10^6#0^6
    println("R = ", R)
    path, gx = shortestPath_BellmanFord(q, arcs, numArcs, origin, destination)
    arcsinPath = findall(path.>0)
    global P_set = [arcsinPath]
global P_pos = []
#     global P_set = [P[1]]

#     global x_now, current_Obj, y_now

#     println("X_feas = ", X_feas)
#     println("Starting M_set ", M_set, ", P_set = ", P_set)

global y_sol = []
#     global 
global numPaths = 1
global newP = true
global newM = true
global iter = 0
#     global theta_now = -1e12
#     global inner_Obj = 1-1e12
conRefNum = 20# max(200,2*length(P_set))
con = Array{JuMP.ConstraintRef}(undef, conRefNum)
global k = 0
global rootOptGap = 0
model = direct_model(Gurobi.Optimizer())
#     set_optimizer_attribute(model, "Presolve", 0)
#     set_optimizer_attribute(model, "Precrush", 1)
#     set_optimizer_attribute(model, "DualReductions", 0)
#     set_optimizer_attribute(model, "PreQLinearize", 0)
#     set_optimizer_attribute(model, "Heuristics", 0)
#     set_optimizer_attribute(model, "MIPGap", 0)
set_optimizer_attribute(model, "LazyConstraints", 1)
# set_optimizer_attribute(model, "OutputFlag", 0)
global loc = "./Documents/GitHub/UncertainTarget/EnumX_RowCol2ndStage/TestInstances/"
# println("Here")
# set_optimizer_attribute(model, "LogFile", loc*"L_"*dataSet*prob*"_"*string(ins)*".log")
# Presolve = 0,
#         PreCrush = 1,
#         Heuristics = 0,

@variable(model, x[1:numArcs], Bin)
@variable(model, theta >= 1-R)#1e6)
@objective(model, Min, theta)
@constraint(model, sum(d_x[a]*x[a] for a=1:numArcs) <= b_x)
    
#     @variable(model, 0 <= x <= 2.5, Int)
#     @variable(model, 0 <= y <= 2.5, Int)
#     @objective(model, Max, y)
#     println("\nM_set = ", M_set," \t P_set = ", P_set)
global last_theta = -1e6 - 1
global start = time()
cb_calls = Cint[]

function my_callback_function(cb_data, cb_where::Cint)
    global numArcs, arcs, d_x, q, b_x, origin, destination, R, d_y, b_y, y_sol
    global newP, newM, P_set, M_set, numPaths,numY, iter, theta_now, inner_Obj
    global k, last_theta
    global P_pos
        # You can reference variables outside the function as normal
        push!(cb_calls, cb_where)
        iter += 1
    
#         println("\niter = ", iter)
#         println("\nM_set = ", M_set," \t P_set = ", P_set)
        # You can select where the callback is run
#         if cb_where != GRB_CB_MIPSOL && cb_where != GRB_CB_MIPNODE
#             return
#         end
        
        # You can query a callback attribute using GRBcbget
        if cb_where == GRB_CB_MIPSOL #|| cb_where == GRB_CB_MIPNODE
            resultP = Ref{Cint}()
            GRBcbget(cb_data, cb_where, GRB_CB_MIPNODE_STATUS, resultP)
#         println(, " ", MOI.get(model, MOI.NodeCount())
#         println("cb_data ", cb_data)
#         println("cb_where ", cb_where)
#         println("GRB_CB_MIPNODE_STATUS ", GRB_CB_MIPNODE_STATUS)   
        # Before querying `callback_value`, you must call:
        Gurobi.load_callback_variable_primal(cb_data, cb_where)
#         if resultP[] != GRB_OPTIMAL
#             return  # Solution is something other than optimal.
#         end
#         if resultP == GRB_OPTIMAL
#             println("\niter = ", iter)
            x_val = callback_value.(Ref(cb_data), x)
            theta_val = callback_value(cb_data, theta)
#             println("x_val = ", findall(x_val.>0))#, "\t theta_val", theta_val)
            P_set, M_set, lambda, pi_, inner_Obj, y_sol = solveLP_findM(x_val)
#             println("M = ", length(M_set), "\t P = ", length(P_set)) 
#             println("P = ", P_set)
#             println("theta_val = ", theta_val, "\tinner_Obj = ", inner_Obj)
#             lambda_pos = findall(lambda.>0)
#             P_pos = P_set[lambda_pos]
#             println("lambda = ", lambda, "\t pi_ = ", pi_)
            numPaths = length(P_set)
            numY = length(M_set)

            TOL = 10^(-3)
#             println("(inner_Obj - theta_val) = ", (inner_Obj - theta_val))
            
#             println("TOL = ", TOL)
            if (inner_Obj - theta_val) > TOL  
                k+=1 
                con_ = @build_constraint(theta >= sum(lambda[i]*(-R)*sum(x[a] for a in P_set[i]) for i=1:numPaths) + pi_)
#                 println("Added con")
#                 println("Type = ", typeof(con_))
#             cut_vio = (sum(lambda[i]*(-R)*sum(x_val[a] for a in P_set[i]) for i=1:numPaths) + pi_) - theta_val
#                 println("Add con : cut violation = ", cut_vio)
#                 println(theta, ">=", sum(lambda[i]*(-R)*sum(x[a] for a in P_set[i]) for i=1:numPaths) + pi_)
                
                w_con = MOI.submit(model, MOI.LazyConstraint(cb_data), con_)
#                 println("Typeof = ", typeof(w_con))
#             MOI.set(model, Gurobi.Lazy(), con[k], 2)
#             MOI.set(model, Gurobi.ConstraintAttribute("lazy"), con, 2)
            end
#         if last_theta > theta_val
#             GRBterminate(backend(model))
#         else
#             last_theta = theta_val
#         end
#         end 
    end
        
       
        
#         if iter > 12
#             # You can terminate the callback as follows:
#             GRBterminate(backend(model))
#         end
    return
end
#     You _must_ set this parameter if using lazy constraints.
    MOI.set(model, MOI.RawParameter("LazyConstraints"), 1)
    MOI.set(model, Gurobi.CallbackFunction(), my_callback_function)
#     println(model)
    optimize!(model)
    
#     println(model)
    totalTime = time() - start
    x_sol = JuMP.value.(x)
    theta_sol = JuMP.value.(theta)
    println("dataSet ", dataSet*prob)
    println("x = ", x_sol)
    println("x_i = ", findall(x_sol.>0))
    println("theta = ", theta_sol)
    println("M_set = ", length(M_set))
    y_pos = findall(y_sol .> 0)
    println("y = ", y_sol[y_pos])
    println("M_pos = ", M_set[y_pos])
    nodeCount = MOI.get(model, MOI.NodeCount())
    println("Node count = ", nodeCount)

int_arc = findall(x_sol.>0)

global myP = []
for n = 1: numPaths #path in P_set
    global myP
    path = P_set[n]
    int_path = intersect(path, int_arc)
    if length(int_path) == 0
        push!(myP,n)
    end
end
P_pos = P_set[myP]
# println("P_pos = ", P_pos)

#Get P_pos
# timesFile = open("./TestInstances/P_"*dataSet*prob*".txt", "a")
# println(timesFile, dataSet*prob, "; Ins ", ins, "; Time ", totalTime, "; theta ", theta_sol,"; x_sol ", findall(x_sol.>0),"; M_select ", M_set[y_pos], "; y_sol ", y_sol[y_pos], "; Iter ", iter, "; M_len ", length(M_set), "; P_len ", length(P_set), "; NodeCount ", nodeCount)
# println(timesFile, "P_pos = ", P_pos)
# close(timesFile)

#Get P_set only
timesFile = open("./TestInstances/P_set_only_"*dataSet*prob*string(ins)*".jl", "a")
println(timesFile, "P_set = ", P_set)
close(timesFile)

Running...
R = 419.71219712404
Academic license - for non-commercial use only - expires 2022-12-04
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 1 rows, 952 columns and 951 nonzeros
Model fingerprint: 0xf73eeadf
Variable types: 1 continuous, 951 integer (951 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [4e+02, 4e+02]
  RHS range        [1e+01, 1e+01]
Presolve time: 0.01s
Presolved: 1 rows, 952 columns, 951 nonzeros
Variable types: 1 continuous, 951 integer (951 binary)

Root relaxation: objective -2.124885e+02, 6 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 -212.48851    0    1          - -212.48851      -     -    2s
H    0     0                       0.9903735 -21

In [158]:
dataSet = "N10_d10_"
ins = 1
prob = ""
global q =[]
include("./TestInstances/"*dataSet*string(ins)*".jl")

for a in [1, 5, 4, 11, 9]
    println(arcs[a, :])
end
# M_pos = Array{Int64,1}[[1], [4], [5, 372], [13, 796], [3, 58], [6], [9], [16, 300], [7, 14, 27], [7, 15, 935], [7, 11], [7, 17, 44], [7, 832], [7, 8, 121], [12, 16], [2, 152], [10, 484, 925]]
# y = [0.0854152, 0.0054319, 0.39549, 0.0395482, 0.00807968, 0.0748314, 0.0125745, 0.00323607, 0.0136343, 0.0419715, 0.0116682, 0.00909369, 0.0320574, 0.0132497, 0.0654111, 0.178472, 0.00983519]
# P_pos = Array{Int64,1}[[7, 463], [4, 355], [1, 113, 355], [10, 697], [8, 652, 697], [11, 750], [2, 164, 750], [3, 306, 750], [6, 384, 750], [12, 777], [13, 796], [17, 935], [3, 58, 295], [2, 74, 148], [9, 74, 656], [2, 103, 149], [6, 103, 373], [2, 146, 162, 653, 781], [11, 146, 746, 781], [6, 146, 384, 746, 781], [15, 186, 832], [16, 186, 883], [2, 162, 235, 641], [11, 20, 235, 746, 778], [6, 20, 235, 384, 746, 778], [11, 146, 745, 774, 781], [6, 146, 384, 745, 774, 781], [11, 20, 235, 745, 774, 778], [6, 20, 235, 384, 745, 774, 778], [2, 153, 293], [9, 293, 659], [6, 355, 378], [11, 355, 736], [2, 164, 355, 736], [6, 355, 384, 736], [2, 162, 355, 653, 784], [11, 355, 746, 784], [6, 355, 384, 746, 784], [6, 146, 387, 917], [11, 355, 745, 774, 784], [6, 355, 384, 745, 774, 784], [6, 355, 385, 837], [11, 355, 748, 837], [6, 355, 384, 748, 837], [9, 669, 750], [9, 103, 657], [9, 146, 669, 746, 781], [9, 20, 235, 669, 746, 778], [9, 146, 669, 745, 774, 781], [9, 355, 669, 736], [9, 355, 669, 746, 784], [14, 146, 830, 917], [9, 355, 669, 745, 774, 784], [9, 355, 669, 748, 837], [14, 44, 74, 830, 916], [16, 103, 882], [14, 20, 235, 818], [16, 293, 886], [16, 355, 888], [14, 23, 355, 818], [16, 78, 146, 882], [16, 85, 355, 882], [16, 239, 355, 884], [16, 281, 420, 886], [16, 262, 443, 885], [16, 282, 443, 886], [16, 261, 394, 443, 885], [16, 258, 300, 394, 443, 885], [15, 45, 109, 235, 850, 916], [12, 44, 74, 774, 779], [12, 103, 774, 780], [12, 44, 74, 776, 830, 916], [12, 103, 776, 829, 882], [12, 146, 774, 781], [12, 146, 776, 830, 917], [13, 44, 74, 779], [13, 103, 780], [13, 146, 781], [13, 186, 793, 829, 883], [8, 44, 74, 653, 779], [8, 235, 641], [17, 44, 74, 916], [15, 293, 834], [17, 45, 109, 235, 916], [3, 44, 74, 295], [15, 355, 837], [2, 168, 186, 918], [2, 162, 355, 655, 837], [12, 257, 293, 766], [12, 420, 768], [12, 262, 443, 766], [14, 103, 829, 882], [14, 150, 189, 235, 819], [2, 167, 355, 888], [2, 23, 147, 355], [13, 355, 784], [3, 146, 297], [14, 78, 146, 829, 882], [17, 146, 917], [8, 18, 106, 146, 640], [14, 150, 189, 221, 355, 819], [2, 44, 74, 168, 916], [2, 162, 652, 697], [2, 146, 168, 917], [1, 122, 697], [2, 18, 107, 147, 186], [2, 78, 146, 149], [1, 121, 652, 697], [1, 74, 105], [1, 103, 121, 653, 780], [1, 103, 124, 829, 882], [1, 106, 146], [8, 443, 644], [14, 80, 196, 443, 829, 882], [12, 379, 394, 443, 767], [1, 121, 463, 645], [16, 263, 463, 885], [1, 127, 935], [16, 283, 528, 886], [2, 85, 149, 355], [2, 162, 221, 355, 641], [2, 162, 463, 645], [16, 78, 136, 463, 882], [3, 186, 310, 832], [16, 619, 891], [3, 186, 311, 918], [8, 528, 647], [2, 78, 136, 149, 463], [1, 127, 463, 925], [1, 127, 186, 918], [1, 66, 105, 463], [16, 89, 528, 882], [1, 52, 114, 357, 528], [2, 88, 149, 463], [2, 157, 528], [12, 411, 497, 528, 768], [2, 162, 528, 647], [2, 89, 149, 528], [14, 830, 935], [1, 127, 380, 528, 923], [2, 78, 135, 149, 380, 528], [1, 30, 104, 619], [2, 150, 202, 593], [2, 162, 619, 650], [12, 619, 770], [2, 162, 497, 528, 646], [15, 420, 840], [15, 443, 841], [15, 262, 443, 833], [15, 282, 443, 834], [15, 394, 443, 839], [15, 323, 394, 443, 836], [3, 309, 830, 935], [13, 793, 830, 935], [3, 310, 850, 935], [13, 794, 850, 935], [13, 20, 235, 778], [15, 379, 394, 443, 838], [8, 146, 655, 850, 917], [3, 103, 296], [10, 694, 777], [10, 103, 676], [10, 103, 695, 780], [10, 46, 146, 675], [10, 48, 186, 675], [10, 20, 235, 674], [10, 45, 107, 186, 675], [10, 45, 109, 235, 675], [10, 46, 130, 235, 675], [8, 103, 653, 780], [15, 146, 850, 917], [17, 186, 918], [6, 384, 745, 777], [16, 257, 293, 885], [16, 257, 281, 420, 885], [16, 488, 889], [16, 257, 283, 528, 885], [17, 130, 235, 917], [14, 58, 830, 916], [6, 385, 463, 842], [11, 463, 748, 842], [6, 384, 463, 748, 842], [4, 343, 384, 745, 777], [4, 343, 385, 463, 842], [15, 463, 842], [9, 186, 669, 748, 850, 918], [16, 349, 593, 888], [7, 456, 697], [7, 355, 456, 681], [3, 306, 745, 777], [3, 74, 306, 729], [2, 164, 186, 745, 776, 830, 918], [2, 164, 186, 748, 850, 918], [2, 20, 147, 235], [7, 18, 113, 355, 454, 640], [7, 18, 113, 355, 458, 778], [7, 454, 652, 697], [7, 457, 777], [7, 74, 444], [7, 103, 455, 657], [7, 186, 446], [3, 299, 355, 362], [7, 355, 458, 784], [7, 186, 455, 669, 748, 832], [13, 355, 794, 837], [2, 151, 239, 355], [7, 355, 455, 672, 888], [7, 18, 113, 355, 460, 818], [7, 45, 113, 355, 460, 830, 916], [7, 45, 113, 355, 455, 669, 748, 850, 916], [7, 420, 455, 662], [7, 146, 457, 774, 781], [7, 78, 146, 455, 657], [7, 77, 107, 186, 455, 657], [10, 355, 681], [7, 77, 113, 355, 455, 657], [7, 355, 460, 829, 888], [7, 77, 113, 355, 460, 829, 882], [7, 239, 355, 448], [7, 189, 235, 447], [7, 355, 457, 776, 829, 888], [7, 18, 113, 355, 457, 776, 818], [7, 77, 113, 355, 457, 776, 829, 882], [7, 239, 355, 457, 776, 829, 884], [7, 150, 189, 218, 239, 355, 445], [7, 257, 293, 457, 766], [7, 420, 457, 768], [7, 262, 443, 457, 766], [7, 379, 394, 443, 457, 767], [7, 450, 497, 528], [3, 186, 309, 830, 918], [7, 61, 239, 355, 444], [3, 48, 186, 295], [7, 452, 593], [7, 457, 619, 770], [2, 18, 109, 147, 235], [2, 150, 189, 235], [2, 150, 195, 420], [2, 155, 443], [2, 150, 196, 443], [2, 45, 109, 168, 235, 916], [2, 153, 281, 420], [2, 152, 262, 443], [2, 153, 282, 443], [2, 154, 394, 443], [2, 152, 263, 463], [2, 156, 488], [2, 153, 283, 528], [2, 156, 475, 528], [2, 156, 474, 497, 528], [2, 154, 396, 593], [2, 160, 619], [2, 154, 397, 619], [2, 155, 435, 619], [2, 154, 394, 435, 619], [2, 161, 639], [2, 155, 437, 697], [2, 154, 394, 437, 697], [2, 154, 399, 715], [2, 150, 197, 463], [2, 155, 433, 593], [2, 154, 394, 433, 593], [2, 163, 728], [2, 164, 745, 777], [17, 58, 916], [15, 58, 850, 916], [15, 850, 935], [16, 173, 235, 883], [16, 258, 303, 528, 885], [16, 324, 420, 887], [16, 475, 528, 889], [16, 474, 497, 528, 889], [16, 477, 593, 889], [16, 542, 639, 890], [16, 478, 697, 889], [16, 479, 715, 889], [16, 323, 394, 443, 887], [16, 546, 728, 890], [16, 542, 633, 728, 890], [16, 478, 692, 728, 889], [16, 476, 545, 697, 889], [16, 476, 545, 692, 728, 889], [16, 652, 697, 892], [16, 482, 817, 889], [16, 654, 817, 892], [16, 481, 792, 817, 889], [16, 484, 881, 889], [16, 894, 915], [8, 653, 793, 830, 935], [16, 74, 479, 698, 889], [16, 103, 464, 889], [16, 486, 889, 935], [16, 103, 481, 780, 889], [16, 74, 894, 895], [16, 146, 894, 897], [16, 48, 186, 831, 893, 936], [16, 107, 186, 894, 896], [16, 235, 894, 900], [16, 830, 893, 935], [16, 20, 235, 818, 893], [16, 186, 830, 893, 918], [16, 894, 914, 935], [16, 109, 235, 894, 896], [16, 186, 894, 914, 918], [16, 45, 107, 186, 831, 893, 936], [16, 45, 109, 235, 831, 893, 936], [16, 130, 235, 894, 897], [2, 74, 164, 729], [16, 74, 831, 893, 937], [16, 46, 146, 831, 893, 936], [16, 46, 130, 235, 831, 893, 936], [16, 545, 697, 890], [16, 480, 765, 795, 881, 889], [16, 654, 815, 892, 915], [16, 480, 765, 792, 817, 889], [16, 74, 654, 799, 892], [16, 485, 889, 915], [16, 74, 480, 752, 889], [16, 74, 485, 889, 895], [16, 479, 712, 792, 817, 889], [16, 653, 795, 881, 892], [16, 146, 654, 815, 892, 897], [16, 186, 655, 832, 892], [16, 146, 480, 753, 889], [16, 186, 465, 889], [16, 186, 486, 889, 918], [2, 103, 164, 730], [2, 156, 477, 593], [2, 155, 432, 563, 593], [2, 154, 394, 432, 563, 593], [2, 154, 400, 728], [2, 154, 403, 777], [2, 155, 439, 817], [2, 154, 394, 439, 817], [2, 166, 881], [2, 154, 405, 915], [2, 156, 486, 935], [2, 74, 158, 529], [2, 61, 148, 239, 355], [2, 103, 156, 464], [2, 103, 161, 620], [2, 103, 158, 542, 620], [2, 103, 154, 398, 657], [3, 18, 107, 186, 294], [3, 20, 235, 294], [2, 146, 164, 746, 781], [2, 74, 154, 398, 656], [2, 18, 107, 164, 186, 746, 778], [3, 20, 235, 306, 746, 778], [3, 186, 310, 850, 918], [3, 20, 235, 309, 818], [11, 745, 777], [15, 44, 74, 850, 916], [9, 669, 745, 777], [3, 299, 372], [17, 48, 186, 916], [6, 387, 935], [6, 186, 387, 918], [11, 45, 107, 186, 745, 776, 830, 916], [3, 45, 107, 186, 295], [6, 109, 235, 374], [13, 103, 793, 829, 882], [13, 293, 793, 829, 886], [13, 132, 293, 781], [13, 355, 793, 829, 888], [13, 48, 186, 779], [13, 18, 107, 186, 778], [13, 23, 355, 778], [13, 45, 113, 355, 779], [13, 18, 113, 355, 778], [13, 20, 221, 355, 778], [13, 21, 239, 355, 778], [14, 153, 293, 819], [9, 77, 109, 235, 657], [6, 45, 109, 235, 387, 916], [6, 77, 107, 186, 373], [6, 77, 109, 235, 373], [11, 74, 729], [6, 74, 384, 729], [11, 103, 730], [6, 103, 384, 730], [3, 299, 369, 796], [16, 235, 485, 889, 900], [16, 20, 235, 483, 818, 889], [16, 109, 235, 485, 889, 896], [16, 45, 109, 235, 480, 751, 889], [16, 45, 109, 235, 486, 889, 916], [16, 293, 466, 889], [3, 45, 109, 235, 295], [3, 77, 107, 186, 296], [3, 77, 109, 235, 296], [3, 130, 235, 297], [13, 58, 779], [9, 103, 672, 882], [9, 257, 293, 658], [9, 61, 239, 355, 656], [9, 420, 662], [9, 443, 663], [16, 653, 796, 892], [11, 745, 774, 796], [17, 293, 922], [14, 363, 563, 593, 820], [12, 776, 830, 935], [1, 114, 369, 796], [2, 162, 652, 694, 777], [8, 58, 653, 779], [3, 306, 745, 774, 796], [15, 173, 235, 832], [1, 122, 694, 777]]

# println(length(M_pos))
# println(length(y))
# println(length(P_pos))

[1, 3]
[3, 2]
[2, 8]
[8, 6]
[6, 10]


In [108]:
theta = 0

numPaths = length(P_pos)
numY = length(M_pos)
for i = 1:numPaths
    path = P_pos[i]
    theta_P = 0
#     println(i, "Path ", path)
    for m = 1:numY
        M = M_pos[m]
#         println("\tM = ", M)
        M_P = intersect(M, path) 
        if length(M_P) > 0
            q_MP = prod((1-q[a]) for a in M_P)
        else
            q_MP = 1
        end
        theta_P = theta_P + q_MP*y[m]
    end
    if theta < theta_P
        theta = theta_P
    end
#     println("\ttheta_P = ", theta_P)
end
println("theta = ", theta)
timesFile = open("./TestInstances/theta_"*dataSet*prob*".txt", "a")
println(timesFile, dataSet*prob, "; Ins ", ins, "; theta ", theta)
close(timesFile)

theta = 0


In [ ]:
length(P_set)

In [ ]:
P_pos = Array{Int64,1}[[7], [3, 52, 64, 68, 109, 190], [5, 90, 154, 190], [1, 37, 48, 90, 190], [2, 41, 59, 76, 128, 190, 193, 204], [2, 42, 59, 90, 131, 190, 204], [1, 50, 76, 136, 190, 193], [5, 76, 116, 128, 155, 190, 193], [2, 51, 59, 76, 173, 190, 205, 226], [1, 50, 64, 122, 135, 190], [5, 41, 64, 122, 152, 190], [2, 59, 64, 167, 190, 208], [6, 42, 56, 76, 130, 159, 190, 193], [3, 20, 76, 110, 162, 190, 210, 231], [4, 76, 136, 190, 193], [4, 90, 123, 135, 190], [5, 56, 76, 130, 156, 190, 193], [2, 59, 76, 190, 193, 209, 222], [4, 43, 76, 133, 190, 193, 222], [3, 30, 64, 67, 92, 109, 167, 190], [3, 34, 67, 76, 109, 173, 190], [4, 41, 64, 122, 133, 190], [4, 42, 90, 131, 133, 190], [6, 26, 76, 158, 173, 190, 226], [3, 26, 76, 110, 158, 173, 190, 226], [3, 41, 76, 110, 128, 159, 190, 193], [3, 76, 110, 162, 173, 190, 214, 226], [1, 51, 76, 173, 190, 226], [4, 12, 76, 119, 135, 173, 190, 226], [5, 41, 76, 128, 152, 190, 193], [2, 11, 58, 64, 81, 167, 190], [5, 20, 76, 150, 190, 231], [2, 8, 14, 58, 76, 81, 184, 190, 193, 209, 222], [2, 8, 14, 58, 76, 81, 173, 184, 190, 209, 223, 238], [4, 64, 122, 135, 190], [6, 41, 64, 122, 159, 190], [3, 42, 52, 64, 110, 130, 159, 190], [3, 29, 56, 76, 110, 163, 190, 193, 229], [2, 41, 59, 64, 122, 190, 204], [6, 76, 145, 162, 190, 193, 213], [3, 76, 110, 145, 162, 190, 193, 213], [3, 56, 68, 76, 109, 190, 193], [1, 51, 76, 145, 190, 193, 225], [4, 25, 76, 120, 135, 190, 193, 222], [3, 41, 90, 110, 123, 159, 190], [2, 10, 58, 81, 90, 154, 190], [4, 20, 35, 76, 136, 190, 191, 231], [4, 76, 137, 173, 190, 217, 238], [1, 50, 76, 128, 135, 190, 193], [4, 37, 90, 137, 190, 215], [4, 64, 92, 137, 167, 190, 216], [4, 90, 131, 137, 190, 217, 237], [2, 16, 46, 58, 82, 90, 190, 210], [3, 52, 62, 68, 90, 109, 131, 190], [5, 16, 90, 150, 190], [4, 37, 76, 88, 98, 115, 136, 173, 190, 191, 223, 238], [3, 41, 64, 110, 122, 159, 190], [4, 26, 76, 120, 135, 173, 190, 226], [6, 41, 76, 128, 159, 190, 193], [2, 43, 59, 76, 173, 190, 204, 223, 238], [3, 32, 67, 76, 109, 148, 190, 231], [2, 45, 58, 64, 82, 122, 190, 196], [6, 42, 90, 131, 159, 190], [1, 39, 48, 64, 122, 190], [5, 17, 56, 76, 130, 150, 190, 193], [5, 76, 118, 155, 173, 190], [4, 76, 128, 135, 190, 193], [4, 42, 56, 76, 130, 133, 190, 193], [1, 20, 35, 48, 76, 190, 231], [2, 20, 59, 76, 96, 190, 207, 231], [3, 64, 71, 109, 122, 190, 196], [2, 20, 46, 58, 76, 82, 190, 210, 231], [5, 27, 29, 56, 76, 151, 190, 193, 229], [4, 25, 41, 76, 120, 133, 190, 193, 222], [5, 18, 76, 145, 150, 190, 193], [6, 26, 76, 145, 158, 190, 193, 225], [3, 26, 76, 110, 145, 158, 190, 193, 225], [1, 18, 35, 48, 76, 145, 190, 193], [6, 18, 76, 145, 162, 190, 193, 210], [3, 18, 76, 110, 145, 162, 190, 193, 210], [2, 12, 58, 76, 81, 173, 190, 226], [6, 16, 90, 162, 190, 210], [3, 72, 76, 109, 173, 190, 238], [2, 10, 20, 58, 76, 81, 150, 190, 231], [2, 20, 45, 58, 76, 82, 125, 150, 190, 196, 231], [6, 20, 76, 162, 190, 210, 231], [2, 46, 58, 76, 82, 173, 190, 214, 226], [3, 37, 41, 90, 110, 128, 159, 190, 191], [4, 37, 90, 128, 135, 190, 191], [3, 8, 14, 37, 70, 81, 90, 109, 179, 190], [4, 37, 90, 136, 190, 191], [3, 37, 55, 68, 90, 109, 179, 190], [3, 90, 110, 143, 162, 190, 213], [4, 20, 76, 125, 135, 150, 190, 231], [4, 16, 90, 125, 135, 150, 190], [4, 16, 40, 46, 90, 133, 190, 210], [6, 90, 143, 162, 190, 213], [6, 76, 162, 173, 190, 214, 226], [4, 37, 42, 56, 90, 130, 133, 190, 191]]
length(P_pos)

In [ ]:
P_u = unique(P_set)